# Exercise 2.2

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import scipy

# Tensorboard for visualizing
from torch.utils.tensorboard import SummaryWriter

### Load dataset

In [6]:
trainset = torchvision.datasets.MNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])

train_loader = torch.utils.data.DataLoader(trainset,
                             batch_size=4, shuffle=True)

val_loader = torch.utils.data.DataLoader(validationset,
                             batch_size=1, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
                             batch_size=1, shuffle=True)

### Model

In [7]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 4)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 4)
        self.fc1 = nn.Linear(16 * 4 * 4, 84)
        #self.fc2 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.leaky_relu(self.fc1(x))
        #x = F.leaky_relu(self.fc2(x))
        x = self.fc2(x)

        return x

model = Net()

# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0002)

writer = SummaryWriter()

### Training

In [8]:
best_accuracy = 0
best_net = 0

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train 1", loss, epoch)
        loss.backward()
        optimizer.step()
        if(i % 100 == 99):
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

    correct = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data
        labels = labels

        outputs = model(inputs)
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train 1", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        best_net = copy.deepcopy(model)
        print(" (new best)")

print('Finished Training')
writer.flush()

Epoch 1 [12000/12000] - Loss: 0.20127514004707336, accuracy:  0.9693333333333334
 (new best)
Epoch 2 [12000/12000] - Loss: 0.06821391731500626, accuracy:  0.979
 (new best)
Epoch 3 [12000/12000] - Loss: 0.05175609141588211, accuracy:  0.9828333333333333
 (new best)
Epoch 4 [12000/12000] - Loss: 0.0037986994720995426, accuracy:  0.9835833333333334
 (new best)
Epoch 5 [12000/12000] - Loss: 0.0045074401423335075, accuracy:  0.9814166666666667
Finished Training


![Picture missing](2_minst_a.png "Validation accuracy each epoch")

### Test

In [9]:
correct = 0
total = 0
for i, data in enumerate(test_loader, 0):
    inputs, labels = data

    outputs = best_net(inputs)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

0.9851


In [10]:
trainset = torchvision.datasets.SVHN('../SVHN/', split='train', download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize(28),
                               torchvision.transforms.Grayscale(1),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

train_len = int(len(trainset) * 0.8)
val_len = len(trainset) - train_len
trainset, validationset = torch.utils.data.random_split(trainset, [train_len, val_len])

train_loader = torch.utils.data.DataLoader(trainset,
                             batch_size=4, shuffle=True)

val_loader = torch.utils.data.DataLoader(validationset,
                             batch_size=1, shuffle=True)


SVHN_loader = torch.utils.data.DataLoader(
  torchvision.datasets.SVHN('../SVHN/', split='test', download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize(28),
                               torchvision.transforms.Grayscale(1),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
                             batch_size=1, shuffle=True)

correct = 0
total = 0
for i, data in enumerate(SVHN_loader, 0):
    inputs, labels = data

    outputs = best_net(inputs)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

Using downloaded and verified file: ../SVHN/train_32x32.mat
Using downloaded and verified file: ../SVHN/test_32x32.mat
0.2609096496619545


### Transfer learning

In [11]:
model_tf = copy.deepcopy(best_net)




for param in model_tf.parameters():
    param.requires_grad = False

#model_tf.fc2 = nn.Linear(84, 10)
for param in model_tf.fc2.parameters():
    param.requires_grad = True

for param in model_tf.fc1.parameters():
    param.requires_grad = True

best_accuracy = 0
#best_net = 0

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model_tf(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train 2", loss, epoch)
        loss.backward()
        optimizer.step()
        if(i % 100 == 99):
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

    correct = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data

        outputs = model_tf(inputs)
        pred = torch.argmax(outputs)

        if pred.numpy() == labels[0].numpy():
            correct += 1
        total += 1
    writer.add_scalar("Validation/train 2", correct/total, epoch)
    print(", accuracy: ", correct/total)
    if correct / total > best_accuracy:
        best_accuracy = correct / total
        #best_net = copy.deepcopy(model)
        print(" (new best)")

print('Finished Training')
writer.flush()

Epoch 1 [14600/14652] - Loss: 2.4346439838409424, accuracy:  0.21792246792246792
 (new best)
Epoch 2 [14600/14652] - Loss: 2.936079978942871, accuracy:  0.21792246792246792
Epoch 3 [14600/14652] - Loss: 3.497138738632202, accuracy:  0.21792246792246792
Epoch 4 [14600/14652] - Loss: 1.8523192405700684, accuracy:  0.21792246792246792
Epoch 5 [14600/14652] - Loss: 3.3787403106689453, accuracy:  0.21792246792246792
Finished Training


![Picture missing](2_minst_b.png "Validation accuracy each epoch")

In [12]:
print(model_tf.fc1.weight)

Parameter containing:
tensor([[ 0.0202, -0.1283, -0.0983,  ..., -0.0219, -0.0420, -0.0269],
        [-0.0104,  0.0457, -0.0931,  ...,  0.0255, -0.0390, -0.0788],
        [ 0.0105, -0.1335, -0.0301,  ...,  0.0994,  0.0706,  0.0285],
        ...,
        [-0.0090, -0.0422,  0.0114,  ...,  0.0603,  0.0755, -0.0604],
        [ 0.0780,  0.1341,  0.1166,  ..., -0.0465,  0.0576,  0.1411],
        [ 0.0461, -0.0297,  0.0133,  ..., -0.0885, -0.0155,  0.0717]],
       requires_grad=True)


In [13]:
print(best_net.fc1.weight)

Parameter containing:
tensor([[ 0.0202, -0.1283, -0.0983,  ..., -0.0219, -0.0420, -0.0269],
        [-0.0104,  0.0457, -0.0931,  ...,  0.0255, -0.0390, -0.0788],
        [ 0.0105, -0.1335, -0.0301,  ...,  0.0994,  0.0706,  0.0285],
        ...,
        [-0.0090, -0.0422,  0.0114,  ...,  0.0603,  0.0755, -0.0604],
        [ 0.0780,  0.1341,  0.1166,  ..., -0.0465,  0.0576,  0.1411],
        [ 0.0461, -0.0297,  0.0133,  ..., -0.0885, -0.0155,  0.0717]],
       requires_grad=True)


In [14]:
correct = 0
total = 0
for i, data in enumerate(SVHN_loader, 0):
    inputs, labels = data

    outputs = model_tf(inputs)
    pred = torch.argmax(outputs)

    if pred.numpy() == labels[0].numpy():
        correct += 1
    total += 1

print(correct/total)

0.2609096496619545
